In [72]:
# Bibliothèques
import pandas as pd
import scipy.sparse
import pickle
import nltk
from nltk.tokenize.treebank import TreebankWordDetokenizer
from bs4 import BeautifulSoup

In [73]:
# fonction pour le TF_IDF_Vectorize
def text_splitter(text):
    return text.split(' ')

In [74]:
# Modèles et données

# tokenizer, lemmatizer and stemmer model
with open('C:/Users/ayoub/OCP6 Data/tokenizer.pk', 'rb') as fp:
    tokenizer = pickle.load(fp)
with open('C:/Users/ayoub/OCP6 Data/lemmatizer.pk', 'rb') as fp:
    lemmatizer = pickle.load(fp)
with open('C:/Users/ayoub/OCP6 Data/p_stemmer.pk', 'rb') as fp:
    p_stemmer = pickle.load(fp)

# NLTK STOPWORDS
engstpw = nltk.corpus.stopwords.words('english')

# bigram and trigram models
with open('C:/Users/ayoub/OCP6 Data/bigram_mod.pk', 'rb') as fp:
    bigram_mod = pickle.load(fp)
with open('C:/Users/ayoub/OCP6 Data/trigram_mod.pk', 'rb') as fp:
    trigram_mod = pickle.load(fp)

# TF_IDF_vectorizer
with open('C:/Users/ayoub/OCP6 Data/Tvectorizer.pk', 'rb') as fp:
    Tvectorizer = pickle.load(fp)

# MultiLabelBinarizer
with open('C:/Users/ayoub/OCP6 Data/mlb_object.pk', 'rb') as fp:
    mlb_object = pickle.load(fp)

# Dimensions reduction
with open('C:/Users/ayoub/OCP6 Data/svdt.sav', 'rb') as fp:
    svdt = pickle.load(fp)

# Classifier
with open('C:/Users/ayoub/OCP6 Data/sgd_clf.sav', 'rb') as fp:
    sgd_clf = pickle.load(fp)


In [75]:
# main function
def predict(question):
    
    q = BeautifulSoup(question, 'lxml').text
    q = tokenizer.tokenize(q.lower())
    q = [token for token in q if token not in engstpw]
    q = [lemmatizer.lemmatize(token) for token in q]
    q = [p_stemmer.stem(token) for token in q]    
    q = trigram_mod[bigram_mod[q]]
    q = Tvectorizer.transform([TreebankWordDetokenizer().detokenize(q)])
    q = svdt.transform(q)
    q = scipy.sparse.csr_matrix(q)
    q = sgd_clf.predict(q)
    q = mlb_object.inverse_transform(q)
    
    return(q)


## test

In [76]:
df1 = pd.read_csv('C:/Users/ayoub/OCP6 Data/ndf1.csv')

In [77]:
question = df1.iloc[1].Title + df1.iloc[1].Body

In [80]:
BeautifulSoup(question, 'lxml').text

"Percentage width child element in absolutely positioned parent on Internet Explorer 7I have an absolutely positioned div containing several children, one of which is a relatively positioned div. When I use a percentage-based width on the child div, it collapses to 0 width on IE7, but not on Firefox or Safari.\nIf I use pixel width, it works. If the parent is relatively positioned, the percentage width on the child works.\n\nIs there something I'm missing here?\nIs there an easy fix for this besides the pixel-based width on the\nchild?\nIs there an area of the CSS specification that covers this?\n\n"

In [79]:
predict(question)

[('css', 'internet-explorer', 'internet-explorer-7')]

In [85]:
df1.iloc[1].Tags

'<html><css><internet-explorer-7>'

In [81]:
question2 = df1.iloc[500].Title + df1.iloc[500].Body

In [82]:
BeautifulSoup(question2, 'lxml').text

'How to pass a comma separated list to a stored procedure?So I have a Sybase stored proc that takes 1 parameter that\'s a comma separated list of strings and runs a query with in in an IN() clause:\nCREATE PROCEDURE getSomething @keyList varchar(4096)\nAS\nSELECT * FROM mytbl WHERE name IN (@keyList)\n\nHow do I call my stored proc with more than 1 value in the list?\nSo far I\'ve tried \nexec getSomething \'John\'         -- works but only 1 value\nexec getSomething \'John\',\'Tom\'   -- doesn\'t work - expects two variables\nexec getSomething "\'John\',\'Tom\'" -- doesn\'t work - doesn\'t find anything\nexec getSomething \'"John","Tom"\' -- doesn\'t work - doesn\'t find anything\nexec getSomething \'\\\'John\\\',\\\'Tom\\\'\' -- doesn\'t work - syntax error\n\nEDIT: I actually found this page that has a great reference of the various ways to pas an array to a sproc\n'

In [83]:
predict(question2)

[('sql', 'sybase-ase')]

In [86]:
df1.iloc[500].Tags

'<sql><sybase-ase>'

In [ ]:
import flask
from flask import request, jsonify

app = flask.Flask(__name__)

# Create some test data for our catalog in the form of a list of dictionaries.
books = [
    {'id': 0,
     'title': 'A Fire Upon the Deep',
     'author': 'Vernor Vinge',
     'first_sentence': 'The coldsleep itself was dreamless.',
     'year_published': '1992'},
    {'id': 1,
     'title': 'The Ones Who Walk Away From Omelas',
     'author': 'Ursula K. Le Guin',
     'first_sentence': 'With a clamor of bells that set the swallows soaring, the Festival of Summer came to the city Omelas, bright-towered by the sea.',
     'published': '1973'},
    {'id': 2,
     'title': 'Dhalgren',
     'author': 'Samuel R. Delany',
     'first_sentence': 'to wound the autumnal city.',
     'published': '1975'}
]


@app.route('/', methods=['GET'])
def home():
    return '''<h1>Distant Reading Archive</h1>
<p>A prototype API for distant reading of science fiction novels.</p>'''


# A route to return all of the available entries in our catalog.
@app.route('/api/v1/resources/books/all', methods=['GET'])
def api_all():
    return jsonify(books)

app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [19/Feb/2020 00:46:35] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [19/Feb/2020 00:47:33] "GET /api/v1/resources/books/all HTTP/1.1" 200 -
